In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def cleanbat(df):
    df['dismissed_by'] = ''
    for x in df.index:
        if df.loc[x, "M/Inns"] =='1st':
            df.loc[x, "M/Inns"] = 1
        elif df.loc[x,'M/Inns'] == '2nd':
            df.loc[x, "M/Inns"] = 2
        elif df.loc[x, 'Posn'] == '-':
            df.drop(x, inplace=True)
        
        if df.loc[x,'How Dismissed'].startswith('b ') or df.loc[x,'How Dismissed'].startswith('c '):
            df.loc[x,'dismissed_by'] = df.loc[x,'How Dismissed'].split('b ')[-1]
        elif df.loc[x,'How Dismissed'].startswith('lbw '):
            df.loc[x,'dismissed_by'] = df.loc[x,'How Dismissed'].split('lbw ')[-1]
        elif df.loc[x,'How Dismissed'].startswith('c&b '):
            df.loc[x,'dismissed_by'] = df.loc[x,'How Dismissed'].split('c&b ')[-1]
        else:
            df.loc[x,'dismissed_by'] = None

        
        runs = str(df.loc[x,'Runs'])
        runs = runs.replace('*','')
        if runs=='-':
            runs = '0'
        # print(runs)
        df.loc[x,'Runs'] = int(runs)
    
    df.drop(['Unnamed: 11','How Dismissed'], axis=1,inplace=True)
    return df

def cleanbowl(df):
    df['dismissed'] = '-'
    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    df.dropna(axis=0, how='all', inplace = True)
    df = df.ffill()
    for x in df.index:
        if df.loc[x, "M/Inns"] =='1st':
            df.loc[x, "M/Inns"] = 1
        elif df.loc[x,'M/Inns'] == '2nd':
            df.loc[x, "M/Inns"] = 2
        if df.loc[x,'Batsman Dismissed'] != np.nan:  
            df.loc[x,'dismissed'] = str(df.loc[x,'Batsman Dismissed']).split(' (')[0]
        else:
            print(df.loc[x,'dismissed'])
            df.loc[x,'dismissed'] = None

    return df

In [3]:
def clean(df, r):
    match r:
        case 'A':
            return cleanbat(df)
        case 'B':
            return cleanbowl(df)

In [4]:


teams = dict()
skipped = []
teams_df = pd.read_excel(r'Z:\DPL\Dataset\DATA PREMIER LEAGUE\Players List.xlsx')
team_list = list(set(teams_df.Team))
for i in team_list:
    teams[i] = {'Batsman' : dict() , 'Bowler' : dict(), 'Allrounder' : dict()}

for x in teams_df.index:
    dirname = r'Z:\DPL\Dataset\DATA PREMIER LEAGUE' + '\\' + teams_df.loc[x,'Team'] + '\\'
    r = ''
    match teams_df.loc[x,'Role'] :
        case 'Batsman':
            dirname+= 'BATSMEN'
            r = 'BATSMEN'
        case 'Allrounder':
            dirname += 'ALLROUNDERS'
            r = 'ALLROUNDERS'
        case 'Bowler':
            dirname+= 'BOWLERS'
            r = 'BOWLERS'

    dirname+='\\'
    dirname +=  teams_df.loc[x,'Team'] + '_' + r + '.xlsx'

    # print(teams_df.loc[x,'Player'])

    if r=='ALLROUNDERS':
        sheetname1 = teams_df.loc[x,'Player']
        try:
            teams[teams_df.loc[x,'Team']][teams_df.loc[x,'Role']][teams_df.loc[x,'Player'] ] = \
                        {
                            'Bat' : clean(pd.read_excel(dirname, sheet_name = teams_df.loc[x,'Player'] + ' a'), 'A') , \
                            'Bowl' : clean(pd.read_excel(dirname, sheet_name = teams_df.loc[x,'Player'] + ' b'),'B')
                        }
        except Exception as e:
            # print(e)
            skipped.append( teams_df.loc[x,'Player'])

    else:
        sheetname = teams_df.loc[x,'Player'] 
        try:
            if r=='BATSMEN':
                teams[teams_df.loc[x,'Team']][teams_df.loc[x,'Role']][teams_df.loc[x,'Player'] ] = clean(pd.read_excel(dirname, sheet_name = sheetname),'A')
            else:
                teams[teams_df.loc[x,'Team']][teams_df.loc[x,'Role']][teams_df.loc[x,'Player'] ] = clean(pd.read_excel(dirname, sheet_name = sheetname), 'B')
                
        except Exception as e:
            # print(e)
            skipped.append( teams_df.loc[x,'Player'])


def parse():
    # print(teams)
    return teams, skipped